In [1]:
import sqlite3
import pandas as pd
import numpy as np
import docx

In [2]:
conn = sqlite3.connect("test.db") 
cursor = conn.cursor()

In [3]:
sql = "SELECT * FROM testidprod WHERE partner is NULL AND state is NULL AND bs=0 AND (factor=1 OR factor=2)"

In [4]:
df = pd.read_sql_query(sql, conn)

In [5]:
df.head()

,id,country,factor,year,res,mir,raw,hash,meta,partner,state,bs
0,1,4,1,2007,1883.000000,1883.0,NaN,bbca52c92d682fd2accfbc25733650bb,"s5:c0803:v1883.0,",None,None,0
1,2,4,1,2008,2019.287306,NaN,NaN,341d23d2a89ce241357e4564aa64bf19,"a1:v32241.5,a23:v2019.29,",None,None,0
2,3,4,1,2009,2165.438781,62600.0,NaN,ce47b1ca9d513dcc48da9401e0dd266e,"s5:c0803:v62600.0,a23:v2165.44,",None,None,0
3,4,4,1,2010,2322.168372,17600.0,NaN,a9fc7d058c9a6d1d5937ee7033663c07,"s5:c0803:v17600.0,a23:v2322.17,",None,None,0
4,5,4,1,2011,2490.241697,46110.0,NaN,3cb16c589b696420a31bb344d49c53a1,"s5:c0803:v46110.0,a23:v2490.24,",None,None,0


In [6]:
df1=pd.pivot_table(df,columns=["factor","year"],values=["res"],aggfunc=np.sum,fill_value=np.nan).reset_index(drop=True)#.rename(columns={'res':'1'})
world={0:df1.sum(axis=1)}
df1=df1.rename(index=world)
df1

factor                          1                                            \
year                         2007          2008          2009          2010   
[455058209116.5964]  1.818306e+10  1.686853e+10  1.833181e+10  1.972401e+10   

factor                                                                       \
year                         2011          2012          2013          2014   
[455058209116.5964]  2.039291e+10  1.958085e+10  2.208439e+10  2.203008e+10   

factor                                           ...             2  \
year                         2015          2016  ...          2010   
[455058209116.5964]  2.170082e+10  2.281889e+10  ...  1.250882e+10   

factor                                                                       \
year                         2011          2012          2013          2014   
[455058209116.5964]  1.371111e+10  1.321901e+10  1.420963e+10  1.537200e+10   

factor                                                                       \
year                         2015          2016          2017          2018   
[455058209116.5964]  1.441794e+10  1.508187e+10  1.537707e+10  1.625388e+10   

factor                             
year                         2019  
[455058209116.5964]  1.651703e+10  

[1 rows x 26 columns]

In [7]:
df2=pd.pivot_table(df,columns=["factor","year"],values=["res"],aggfunc=np.sum,fill_value=np.nan).reset_index(drop=True)#.rename(columns={'res':'1'})
df2=df2[1].join(df2[2],rsuffix='_r', lsuffix='_l').join(df2[2]/df2[1],rsuffix='_r', lsuffix='_l').rename(index=world)
df2.columns = pd.MultiIndex.from_tuples(zip([1]*13+[2]*13+[6]*13, pd.unique(df['year']).tolist()*3),names=['factor', 'year'])
df2

factor                          1                                            \
year                         2007          2008          2009          2010   
[455058209116.5964]  1.818306e+10  1.686853e+10  1.833181e+10  1.972401e+10   

factor                                                                       \
year                         2011          2012          2013          2014   
[455058209116.5964]  2.039291e+10  1.958085e+10  2.208439e+10  2.203008e+10   

factor                                           ...         6            \
year                         2015          2016  ...      2010      2011   
[455058209116.5964]  2.170082e+10  2.281889e+10  ...  0.634192  0.672347   

factor                                                                 \
year                     2012      2013      2014      2015      2016   
[455058209116.5964]  0.675099  0.643424  0.697773  0.664396  0.660938   

factor                                           
year                     2017    2018      2019  
[455058209116.5964]  0.662354  0.6705  0.647046  

[1 rows x 39 columns]

In [8]:
df2.to_excel("report.xlsx")  

In [9]:
def cagr(start_value, end_value, num_periods=2):    
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

In [10]:
df6=df2[6].iloc[[0]].to_numpy().tolist()
df6 = pd.DataFrame(df6,columns =[i for i in range(13)])
df6 = df6.transpose()
df6[1]=cagr(df6-df6.diff().fillna(df6),df6)
df6.loc[0,1]=df6.loc[12,1]=np.nan
df6[0]=pd.unique(df['year']).tolist()
df6.columns=['Year', 'World Value']
dfp=pd.DataFrame(df6)
df6.index=pd.MultiIndex.from_tuples(zip([6]*13),names=['Factor'])
df6

,Year,World Value
Factor,,
6,2007,NaN
6,2008,0.246859
6,2009,-0.045870
6,2010,-0.034409
6,2011,0.060162
6,2012,0.004093
6,2013,-0.046919
6,2014,0.084468
6,2015,-0.047834


In [11]:
doc = docx.Document()
doc.add_heading('Calculating CAGR', 0)
t = doc.add_table(df6.shape[0]+2, df6.shape[1]+1)
for j in range(df6.shape[-1]):
    t.cell(0,j+1).text = df6.columns[j]
for i in range(df6.shape[0]):
    t.cell(i+1,0).text = str(df6.index[i][0])
for i in range(df6.shape[0]):
    for j in range(df6.shape[-1]):
        t.cell(i+1,j+1).text = str(df6.values[i,j])
t.cell(0,0).text = "Factor"   
cr=round(cagr(df6.values[1,1],
        df6.values[df6.shape[0]-2,1],
        len(pd.unique(df['year']).tolist())-2),2)
if cr>0: 
    factor="grew" 
else: 
    factor="decreased"
doc.add_paragraph(f'Factor 6 {factor} by avg {cr}% every year from {int(df6.values[1,0])} to {int(df6.values[11,0])}')
doc.save('report.docx')

In [ ]:
#   Возможные варианты решения:
#
#   Вар2 - выполнить преобразования столбцов через циклы
#   Вар3 - через запросы SQL, с последующим доведением в python.